# Анализ двустороннего экспоненциального распределения
Импорт необходимых библиотек и настройка параметров

In [ ]:
import numpy as np
from scipy import stats
from scipy.optimize import Bounds, minimize

# Задаем параметры
n = 1000  # размер выборки
lambda_true = 2  # истинное значение параметра λ
a = 0  # параметр сдвига
alpha = 4  # известный параметр

# Генерация выборки из двустороннего экспоненциального распределения
np.random.seed(42)
u = np.random.uniform(0, 1, n)
X = np.zeros(n)

for i in range(n):
    if u[i] < 0.5:
        X[i] = a - (1/lambda_true) * np.log(2*u[i])
    else:
        X[i] = a + (1/lambda_true) * np.log(2*(1-u[i]))

## 1.1 Числовые характеристики модели
Вычисление математического ожидания и дисперсии

In [ ]:
# Теоретические значения
theoretical_mean = a
theoretical_variance = 2 / (lambda_true**2)

print(f"Теоретическое математическое ожидание: {theoretical_mean}")
print(f"Теоретическая дисперсия: {theoretical_variance}")

# Выборочные значения
sample_mean = np.mean(X)
sample_variance = np.var(X)

print(f"\nВыборочное среднее: {sample_mean:.4f}")
print(f"Выборочная дисперсия: {sample_variance:.4f}")

## 1.2 Оценка параметра λ методом максимального правдоподобия

In [ ]:
# Функция правдоподобия (отрицательный логарифм)
def neg_log_likelihood(omega):
    # Вычисление первого терма
    term1 = -n * np.log(omega)

    # Вычисление второго терма
    term2 = -n * np.log(2)

    # Вычисление третьего терма
    term3 = omega * np.sum(np.abs(X - a))

    # Суммируем все термы
    # Используем item() для получения скалярного значения
    result = (term1 + term2 + term3).item()

    return result





# Начальное приближение на основе метода моментов
initial_omega = np.sqrt(2 / sample_variance)

# Поиск оценки максимального правдоподобия с ограничениями
bounds = [(0.1, 10.0)]
result = minimize(neg_log_likelihood,
                 x0=initial_omega,
                 method='L-BFGS-B',
                 bounds=bounds,
                 options={'disp': True})

omega_mle = result.x[0]


print(f"Начальное приближение ω (метод моментов): {initial_omega:.4f}")
print(f"Оценка ω методом максимального правдоподобия: {omega_mle:.4f}")

## 1.3 Асимптотический доверительный интервал

In [ ]:
# Вычисление асимптотического доверительного интервала
fisher_info = n / (omega_mle**2)
std_error = np.sqrt(1/fisher_info)

# Доверительный интервал (95%)
z_score = stats.norm.ppf(0.975)
ci_lower = omega_mle - z_score * std_error
ci_upper = omega_mle + z_score * std_error

print(f"95% доверительный интервал для ω: ({ci_lower:.4f}, {ci_upper:.4f})")